## UH Workshop
### CNN sample code 
(Based on: Torres and Cantu, 2019)

In [2]:
# Larger Convolutional Neural Network for MNIST

#We import classes and function then load and prepare the data the same as in the previous CNN example.

# CNN for the MNIST Dataset
import numpy
import os
import getopt
import sys
import random
import cv2
import scipy

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Input
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as K
from PIL import Image
from sklearn.utils import shuffle

K.set_image_dim_ordering('th')

num_classes = 10



Using TensorFlow backend.


We first try the model using the standard MNIST data.

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]


We define a large CNN architecture with two convolutional layers, two max pooling layers, and two fully connected layers. The network topology can be summarized as follows:

- Zero padding of size 2
- Convolutional layer with 32 feature maps of size 4×4.
- Pooling layer taking the max over 2*2 patches.
- Convolutional layer with 32 feature maps of size 3×3.
- Pooling layer taking the max over 2*2 patches.
- Dropout layer with a probability of 20%.
- Flatten layer.
- Fully connected layer with 128 neurons and rectifier activation.
- Fully connected layer with 50 neurons and rectifier activation.
- Output layer.

In [4]:

# define the larger model
def larger_model():
# create model
    model = Sequential()
    model.add(ZeroPadding2D(padding=(2, 2), input_shape=(1,28,28), data_format=None))
    model.add(Conv2D(32, (4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
# Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

You will see that the training process takes some time. We illustrate the model using only one epoch, but the optimal training with these data is with about 10 epocs. You can just change the value for epochs in model.fit()

In [5]:
# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 171s 3ms/step - loss: 0.3246 - acc: 0.9024 - val_loss: 0.0804 - val_acc: 0.9741
Large CNN Error: 2.59%


In [ ]:
#Save the model weights
model.save_weights('.../MNIST_weights')


The cell below resizes the images to 28x28. CNN models work better with squared images.

In [ ]:
#Resize the images to 28x28 and import them to a new directory

path_from='/Users/franciscocantu/Dropbox/UH/Teaching/Workshop/Data/Digits/'
path_to='/Users/franciscocantu/Dropbox/UH/Teaching/Workshop/Data/DigitsResized/'


dirs_from = os.listdir( path_from )

#I make sure to leave outside hidden files
dirs_to  = []
for item in os.listdir(path_to):
    if not item.startswith('.'):
        dirs_to.append(item)
        

for i in range(len(dirs_from)):
    dirpath = path_from+dirs_from[i]
    files = os.listdir(path_from+dirs_from[i])
    for j in range(1,len(files)):
        filepath = path_from+dirs_from[i]+"/"+files[j]
        if ".jpg" in filepath:
            im = Image.open(filepath)
            f, e = os.path.splitext(filepath)
            imResize = im.resize((28,28), Image.ANTIALIAS)
            filepath1= path_to+dirs_to[i]+"/"+files[j]
            g, h = os.path.splitext(filepath1)
            imResize.save(g + 'R.jpg', 'JPEG', quality=95)

The cell below randomly picks the examples for the validity set

In [ ]:
# Assign a random sample (10%) of images to the validity set
path_from='/Users/franciscocantu/Dropbox/UH/Teaching/Workshop/Data/DigitsResized/'
path_to='/Users/franciscocantu/Dropbox/UH/Teaching/Workshop/Data/DigitsValidation/'

dirs_from  = []
for item in os.listdir(path_from):
    if not item.startswith('.'):
        dirs_from.append(item)

dirs_to  = []
for item in os.listdir(path_to):
    if not item.startswith('.'):
        dirs_to.append(item)
N = 100
P = 0.10


for i in range(len(dirs_from)):
    dirpathO = path_from+dirs_from[i]
    dirpathD = path_to+dirs_to[i]
    j = 0
    while j < (P*len(os.listdir(dirpathO))):
        picked = random.choice(os.listdir(dirpathO))
        fO = os.path.join(dirpathO, picked)
        fD = os.path.join(dirpathD, picked)
        os.rename(fO, fD)
        j = j + 1
 

In [ ]:
#To do list: Clean this function. I am tired, sick, and lazy right now to think in loops
#Transforming the images to 2D

def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    
    digits=['0','1','2','3','4','5','6','7','8','9']

    for digit in os.listdir(folder):
        if not digit.startswith('.'):
            for x in range(10):
                if digit in digits[x]:
                    label = digits[x]
                    
            for image_filename in os.listdir(folder + digit):
                img_file = cv2.imread(folder + digit + '/' + image_filename, 0)
                if img_file is not None:
                    img_arr = numpy.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = numpy.asarray(X)
    y = numpy.asarray(y)
    return X,y

In [ ]:
#A model only using our data

def tuned_model():
# create model
    model = Sequential()
    model.add(ZeroPadding2D(padding=(2, 2), input_shape=(1,28,28), data_format=None))
    model.add(Conv2D(32, (4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.30))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
 
#To load MNIST weights and freeze the first set of layers, uncomment the lines below:
    #model.load_weights('/Users/franciscocantu/Dropbox/VisualAnalysis/Methods Article/Data/MNIST_weights')
    #    model.layers
    #for layer in model.layers[:3]:
    #    layer.trainable = False

# Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [ ]:
X_train1, y_train1 = get_data('/Users/franciscocantu/Dropbox/UH/Teaching/Workshop/Data/DigitsResized/')
X_test1, y_test1 = get_data('/Users/franciscocantu/Dropbox/UH/Teaching/Workshop/Data/DigitsValidation/')

nsamples, nx, ny = X_train1.shape
nsamples2, nx2, ny2 = X_test1.shape
X_train1 = X_train1.reshape(X_train1.shape[0], 1, 28, 28).astype('float32')
X_test1 = X_test1.reshape(X_test1.shape[0], 1, 28, 28).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train1 = X_train1 / 255
X_test1 = X_test1 / 255
# one hot encode outputs
y_train1 = np_utils.to_categorical(y_train1)
y_test1 = np_utils.to_categorical(y_test1)
num_classes = y_test1.shape[1]

In [ ]:
X_test1.shape

In [ ]:
X_train1.shape

We add to our model data augmentation, this create artifically cases from our original database.

In [ ]:
# build the model
model = tuned_model()


# Data augmentation

X_train1, y_train1 = shuffle(X_train1, y_train1)
X_test1, y_test1 = shuffle(X_test1, y_test1)

imgen_train = ImageDataGenerator( #uncomment the two lines below for data augmentation
    zoom_range=0.15, #zoom in and out                           
    rotation_range=20 #Degree of random rotations
)

imgen_valid = ImageDataGenerator()

imgen_train.fit(X_train1)
imgen_valid.fit(X_test1)

Again, we illustrate the model using only two epochs. You will find that the predictions of the model with these data are far from optimal. Our tests show that we should stop training this model after about 100 epochs, so don't hold your breath.

In [1]:
# Fit the model
cnn = model.fit_generator(imgen_train.flow(*shuffle(X_train1, y_train1), batch_size=250), 
                epochs=100,
                steps_per_epoch=250,
                validation_data=(X_test1, y_test1))

NameError: name 'model' is not defined

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test1, y_test1, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

Y_pred = model.predict(X_test1,verbose=2)
y_pred = numpy.argmax(Y_pred,axis=1)

# Predictions per category
for i in range(10):
    print(i, confusion_matrix(numpy.argmax(y_test1,axis=1), y_pred)[i].sum())

# Confusion matrix
print(confusion_matrix(numpy.argmax(y_test1,axis=1), y_pred))



In [ ]:
# Prediction probabilities
prediction_prob=pd.DataFrame(numpy.amax(Y_pred, axis=1))
label=pd.DataFrame(numpy.argmax(y_test1,axis=1))
y_pred=pd.DataFrame(y_pred)

test = pd.concat([label, y_pred, prediction_prob], axis=1)
test.columns = ['label','predicted','prob']
test['flagged'] = numpy.where(test['prob']<.8, '1', '0')

print(test)

In [ ]:
#Performance plots

plt.figure(0)
plt.plot(cnn.history['acc'], 'r')
plt.plot(cnn.history['val_acc'], 'g')
plt.xticks(numpy.arange(0, 25, 5.0))
plt.rcParams['figure.figsize'] = (8,6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train', 'val'])

plt.figure(1)
plt.plot(cnn.history['loss'], 'r')
plt.plot(cnn.history['val_loss'], 'g')
plt.xticks(numpy.arange(0, 25, 5.0))
plt.rcParams['figure.figsize'] = (8,6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'val'])

plt.show()

In [ ]:
#Save the model weights
model.save_weights('.../Data/Tuned_weights')

In [ ]:
model.summary()